In [14]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [15]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf

In [16]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
                  x.q_id \
                , x.q_posttypeid \
                , x.q_acceptedanswerid \
                , x.q_parentid \
                , x.q_creationdate \
                , x.q_score \
                , x.q_viewcount \
                , x.q_owneruserid \
                , x.q_title \
                , x.q_tags \
                , x.q_answercount \
                , x.q_commentcount \
                , x.q_reputation \
                , b.id               as a_id \
                , b.posttypeid       as a_posttypeid \
                , b.acceptedanswerid as a_acceptedanswerid \
                , b.parentid         as a_parentid \
                , b.creationdate     as a_creationdate \
                , b.score            as a_score \
                , b.viewcount        as a_viewcount \
                , b.owneruserid      as a_owneruserid \
                , b.title            as a_title \
                , b.tags             as a_tags \
                , b.answercount      as a_answercount \
                , b.commentcount     as a_commentcount \
                , d.reputation       as a_reputation \
  from ( \
           select a.id               as q_id \
                , a.posttypeid       as q_posttypeid \
                , a.acceptedanswerid as q_acceptedanswerid \
                , a.parentid         as q_parentid \
                , a.creationdate     as q_creationdate \
                , a.score            as q_score \
                , a.viewcount        as q_viewcount \
                , a.owneruserid      as q_owneruserid \
                , a.title            as q_title \
                , a.answercount      as q_answercount \
                , a.commentcount     as q_commentcount \
                , c.reputation       as q_reputation \
                , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags \
           from posts a \
              , users c \
           where a.creationdate >= '2023-08-03' \
             and a.posttypeid = '1' \
             and a.owneruserid is not null \
             and c.id = a.owneruserid \
             and (replace(replace(lower(a.tags), '<', ''), '>', ' ') like '%java %' or \
                  replace(replace(lower(a.tags), '<', ''), '>', ' ') like '%spring%') \
       )   x \
        , posts b \
        , users d \
where b.parentid = x.q_id \
  and b.posttypeid = '2' \
  and b.owneruserid is not null \
  and d.id = b.owneruserid \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [17]:
df = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_reputation' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_reputation' 
])

In [18]:
df.head()

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_parentid,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation
0,77579473,1,NaN,None,2023-11-30 14:58:18.637,1,68,20042929,Java generics and collections: Why it is possi...,java generics collections,...,77579473,2023-11-30 15:06:10.277,3,None,1180351,None,None,None,0,7014
1,77556100,1,77579537.0,None,2023-11-27 10:18:06.533,1,75,5057736,Problem reading classes from JPMS module in Sp...,java spring classloader java-module spring-con...,...,77556100,2023-11-30 15:07:42.510,0,None,5057736,None,None,None,0,11081
2,77577380,1,NaN,None,2023-11-30 09:43:51.573,0,33,10804494,Not able to mock Logger,spring-boot unit-testing logging mockito slf4j,...,77577380,2023-11-30 15:15:39.603,0,None,21237557,None,None,None,2,194
3,77579068,1,NaN,None,2023-11-30 14:01:12.353,-1,66,22920096,What's the reason to call/create non-default c...,java constructor,...,77579068,2023-11-30 14:05:45.183,1,None,2422776,None,None,None,0,299523
4,77579473,1,NaN,None,2023-11-30 14:58:18.637,1,68,20042929,Java generics and collections: Why it is possi...,java generics collections,...,77579473,2023-11-30 15:23:46.337,2,None,2402272,None,None,None,0,163152


In [19]:
print(df['q_creationdate'].min())
print(df['a_creationdate'].max())

2023-08-03 01:04:49.693000
2023-12-03 10:02:09.807000


In [20]:
df_gephi = df[['q_owneruserid', 'a_owneruserid']]

In [21]:
df_gephi.columns = ['Source', 'Target']

In [22]:
df_gephi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12428 entries, 0 to 12427
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Source  12428 non-null  int64
 1   Target  12428 non-null  int64
dtypes: int64(2)
memory usage: 194.3 KB


In [23]:
np.append(df_gephi['Source'].values, df_gephi['Target'].values)

array([20042929,  5057736, 10804494, ..., 23014478, 20347223, 22778561])

In [24]:
node = np.unique(np.append(df_gephi['Source'].values, df_gephi['Target'].values))

In [25]:
# 질문자->답변자 로 edgelist 생성
edgelist = list(zip(df_gephi['Source'], df_gephi['Target']))

In [26]:
df_gephi.to_csv('../../data/gephi.csv', index=False)

In [27]:
print(df['q_owneruserid'].min())
print(df['q_owneruserid'].max())

214
23031022


In [28]:
df_question = df_gephi['Source'].value_counts().reset_index()
df_answer = df_gephi['Target'].value_counts().reset_index()

In [29]:
df_gephi

,Source,Target
0,20042929,1180351
1,5057736,5057736
2,10804494,21237557
3,22920096,2422776
4,20042929,2402272
...,...,...
12423,23013595,507738
12424,15441,12658226
12425,12067117,23014478
12426,21617245,20347223


In [30]:
df_answer = df_answer.rename(columns = {'Target': 'user_id', 'count':'answer_cnt'})
df_question = df_question.rename(columns = {'Source': 'user_id', 'count':'question_cnt'})

In [31]:
df_qna = pd.merge(df_question, df_answer, on = 'user_id', how='outer')

In [32]:
df_qna.fillna(0, inplace=True)

In [33]:
df_qna

,user_id,question_cnt,answer_cnt
0,22186847,32.0,11.0
1,352290,24.0,0.0
2,4450024,22.0,0.0
3,11873562,22.0,7.0
4,4260303,21.0,0.0
...,...,...,...
11387,22756364,0.0,1.0
11388,22752098,0.0,1.0
11389,1153026,0.0,1.0
11390,22756343,0.0,1.0


In [34]:
# user별 question/answer count를 가지고 있는 데이터
with open('../../data/df_qna.pkl', 'wb') as f:
    pickle.dump(df_qna, f)


In [35]:
# gephi 시각화용 데이터 
with open('../../data/df_gephi.pkl', 'wb') as f:
    pickle.dump(df_gephi, f)

In [36]:
# 전체 데이터셋 
with open('../../data/df.pkl', 'wb') as f:
    pickle.dump(df, f)